# Map Servers Assignment – Part 1 & 2

This notebook implements the deliverables for the map server assignment. The first part summarises the Model Context Protocol (MCP) concepts and patterns observed in existing map servers, while the second part implements custom map servers using the OpenAI Agents SDK.


In [3]:
import os
import openai
from google.colab import userdata

# Configure the OpenAI API key from a Colab secret named 'KEY'
# In Google Colab, add your API key via Settings → Secrets and name it 'KEY'.

# Try to get the key from Colab secrets
api_key = userdata.get('KEY')

if not api_key:
    raise RuntimeError('Missing KEY in Colab secrets. Please add your OpenAI key as a secret named "KEY".')

os.environ['OPENAI_API_KEY'] = api_key
openai.api_key = api_key
print('OpenAI API key has been configured.')

OpenAI API key has been configured.


In [5]:
!pip install agents

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 753.1/753.1 kB 30.4 MB/s eta 0:00:00
  Created wheel for agents: filename=agents-1.4.0-py3-none-any.whl size=62714 sha256=bc68bfdb7f4ce2e1e7f7c968b9350d93a5c63267addab1944572a4b31ad9a9dc
  Stored in directory: /root/.cache/pip/wheels/e5/82/e5/2790dbbc1ad6037f1001bc436ea963e0877fff918dddc74fe2
Successfully built agents


In [2]:
# Uninstall current TensorFlow version
!pip uninstall tensorflow -y

# Install TensorFlow 1.x (e.g., 1.15.2) and tensorflow-compat-v1
!pip install tensorflow==1.15.2 tensorflow-compat-v1

print("TensorFlow downgrade initiated. Please restart the Colab runtime (Runtime > Restart runtime) for changes to take effect.")

Found existing installation: tensorflow 2.19.0
Uninstalling tensorflow-2.19.0:
  Successfully uninstalled tensorflow-2.19.0
ERROR: Could not find a version that satisfies the requirement tensorflow==1.15.2 (from versions: 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0, 2.18.1, 2.19.0rc0, 2.19.0, 2.19.1, 2.20.0rc0, 2.20.0)
ERROR: No matching distribution found for tensorflow==1.15.2
TensorFlow downgrade initiated. Please restart the Colab runtime (Runtime > Restart runtime) for changes to take effect.


In [2]:
# 1) Make sure we do NOT use the wrong "agents" package
!pip uninstall -y agents   # this removes the RL library that wants TensorFlow
!pip install -U openai-agents httpx openai


Found existing installation: agents 1.4.0
Uninstalling agents-1.4.0:
  Successfully uninstalled agents-1.4.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.8/231.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.4/463.4 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 65.2 MB/s eta 0:00:00
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.33.2
    Uninstalling pydantic_core-2.33.2:
      Successfully uninstalled pydantic_core-2.33.2
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.11.10
    Uninstalling pydantic-2.11.10:
      Successfully uninstalled pydantic-2.11.10
  Attempting uninstall: openai
    Found existing installation: o

In [2]:
import os
import openai
from google.colab import userdata

# Configure the OpenAI API key from a Colab secret named 'KEY'
# In Google Colab, add your API key via Settings → Secrets and name it 'KEY'.

# Try to get the key from Colab secrets
api_key = userdata.get('KEY')

if not api_key:
    raise RuntimeError('Missing KEY in Colab secrets. Please add your OpenAI key as a secret named "KEY".')

os.environ['OPENAI_API_KEY'] = api_key
openai.api_key = api_key
print('OpenAI API key has been configured.')

OpenAI API key has been configured.


In [3]:
from agents import Agent, Runner, function_tool
import httpx
import asyncio


In [4]:
# ---------- Map Server 1: Geocoding & POI search (Nominatim) ----------

@function_tool
async def geocode_address(address: str, limit: int = 3) -> list:
    """
    Perform forward geocoding using OpenStreetMap's Nominatim API.

    Parameters:
        address (str): The address or place name to search for.
        limit (int): Maximum number of candidate results to return (default 3).

    Returns:
        List[dict]: A list of dicts containing latitude, longitude and display name.
    """
    params = {
        'q': address,
        'format': 'json',
        'limit': limit
    }
    async with httpx.AsyncClient() as client:
        response = await client.get('https://nominatim.openstreetmap.org/search', params=params, headers={'User-Agent': 'map-assistant'})
        response.raise_for_status()
        results = response.json()
    return [
        {
            'lat': float(item['lat']),
            'lon': float(item['lon']),
            'display_name': item['display_name']
        }
        for item in results
    ]


@function_tool
async def reverse_geocode(lat: float, lon: float, zoom: int = 18) -> dict:
    """
    Reverse geocode a latitude/longitude pair using Nominatim.

    Parameters:
        lat (float): Latitude of the point.
        lon (float): Longitude of the point.
        zoom (int): Level of detail for the address (0–18).

    Returns:
        dict: A dict containing the address string and bounding box for the result.
    """
    params = {
        'lat': lat,
        'lon': lon,
        'format': 'json',
        'zoom': zoom
    }
    async with httpx.AsyncClient() as client:
        response = await client.get('https://nominatim.openstreetmap.org/reverse', params=params, headers={'User-Agent': 'map-assistant'})
        response.raise_for_status()
        result = response.json()
    return {
        'display_name': result.get('display_name'),
        'lat': float(result.get('lat', lat)),
        'lon': float(result.get('lon', lon)),
        'boundingbox': result.get('boundingbox')
    }


@function_tool
async def search_poi(query: str, city: str, limit: int = 5) -> list:
    """
    Search for points of interest (POI) in a given city using Nominatim.

    Parameters:
        query (str): What you are searching for (e.g. 'coffee shop', 'library').
        city (str): Name of the city to restrict the search to.
        limit (int): Maximum number of results to return (default 5).

    Returns:
        List[dict]: A list of dicts with name, latitude and longitude of matching POIs.
    """
    # Combine query with the city name to improve search relevance
    full_query = f"{query}, {city}"
    params = {
        'q': full_query,
        'format': 'json',
        'limit': limit
    }
    async with httpx.AsyncClient() as client:
        response = await client.get('https://nominatim.openstreetmap.org/search', params=params, headers={'User-Agent': 'map-assistant'})
        response.raise_for_status()
        results = response.json()
    return [
        {
            'name': item.get('display_name'),
            'lat': float(item['lat']),
            'lon': float(item['lon'])
        }
        for item in results
    ]



In [5]:
# ---------- Map Server 2: Routing & Tile service (OSRM + OSM tiles) ----------

@function_tool
async def plan_route(origin_lat: float, origin_lon: float, dest_lat: float, dest_lon: float, profile: str = 'driving') -> dict:
    """
    Plan a route between an origin and destination using the OSRM demo server.

    Parameters:
        origin_lat (float): Latitude of the starting point.
        origin_lon (float): Longitude of the starting point.
        dest_lat (float): Latitude of the destination.
        dest_lon (float): Longitude of the destination.
        profile (str): Travel mode (driving, walking, cycling). Default 'driving'.

    Returns:
        dict: A dictionary containing the distance (km), duration (minutes) and an encoded polyline for the route.
    """
    coords = f"{origin_lon},{origin_lat};{dest_lon},{dest_lat}"
    url = f"https://router.project-osrm.org/route/v1/{profile}/{coords}"
    params = {
        'overview': 'full',
        'geometries': 'polyline',
        'steps': True
    }
    async with httpx.AsyncClient() as client:
        response = await client.get(url, params=params)
        response.raise_for_status()
        data = response.json()
    route = data['routes'][0]
    return {
        'distance_km': route['distance'] / 1000.0,
        'duration_min': route['duration'] / 60.0,
        'polyline': route['geometry'],
        'legs': route['legs']
    }


@function_tool
def tile_url(z: int, x: int, y: int) -> str:
    """
    Construct a URL for an OpenStreetMap raster tile.

    Parameters:
        z (int): Zoom level.
        x (int): Tile column.
        y (int): Tile row.

    Returns:
        str: URL pointing to a PNG tile on the OSM tile server.
    """
    return f"https://tile.openstreetmap.org/{z}/{x}/{y}.png"


@function_tool
def list_basemap_styles() -> list:
    """
    List available raster or vector basemap styles.

    Returns:
        list: Example basemap style descriptors for use with client libraries like MapLibre.
    """
    return [
        {
            'name': 'OSM Standard',
            'type': 'raster',
            'tile_url': 'https://tile.openstreetmap.org/{z}/{x}/{y}.png'
        },
        {
            'name': 'OSM Bright (Mapbox GL style)',
            'type': 'vector',
            'style_url': 'https://demotiles.maplibre.org/style.json'
        },
        {
            'name': 'Carto Voyager',
            'type': 'raster',
            'tile_url': 'https://cartodb-basemaps-a.global.ssl.fastly.net/rastertiles/voyager/{z}/{x}/{y}.png'
        }
    ]



In [33]:
# ---------- Agent wiring and helper ----------

# Create the map assistant agent with all tools registered
map_agent = Agent(
    name='MapAssistant',
    instructions=(
        'You are a helpful map assistant. Use the available tools to perform '
        'geocoding, reverse geocoding, searching for points of interest, planning routes, '
        'retrieving map tiles and listing basemap styles. For routing, prefer the OSRM '
        'demo server. For geocoding and POI search use the Nominatim API. Always include '
        'units (kilometres, minutes) when returning distances or durations.'
    ),
    tools=[
        geocode_address,
        reverse_geocode,
        search_poi,
        plan_route,
        tile_url,
        list_basemap_styles
    ]
)


In [34]:
import json

async def ask_map_agent_with_details(question: str):
    result = await Runner.run(map_agent, input=question)

    print("=== Final answer ===")
    print(result.final_output)
    print()

    print("=== Tool calls & results (new_items) ===")
    for idx, item in enumerate(result.new_items, start=1):
        raw = item.raw_item
        if hasattr(raw, "model_dump"):
            data = raw.model_dump(exclude_unset=True)
        elif isinstance(raw, dict):
            data = raw
        else:
            data = str(raw)

        print(f"\n--- Item {idx} ({type(item).__name__}) ---")
        print(json.dumps(data, indent=2, ensure_ascii=False))



In [37]:
await ask_map_agent_with_details(
    "Search for at least four coffee shops in Hamra, Beirut and then give me a step by step driving route between the first two."
)


=== Final answer ===
Here are four coffee shops in Hamra, Beirut:

1. Urbanista Hamra — شارع المهتما غاندي
2. Cafe Younes Hamra — شارع بعلبك
3. Café Hamra — Hamra Street
4. Starbucks Hamra — شارع المقدسي

Driving Route from Urbanista Hamra to Cafe Younes Hamra:

1. Start from Urbanista Hamra on شارع المهتما غاندي.
2. Head south on شارع المهتما غاندي for about 80 metres.
3. Turn left onto شارع بعلبك and continue for about 420 metres.
4. Turn left onto شارع 67 and follow it for about 67 metres.
5. Arrive at Cafe Younes Hamra.

- Total distance: 0.57 km
- Estimated driving time: approximately 1.5 minutes

Let me know if you want a map or more details!

=== Tool calls & results (new_items) ===

--- Item 1 (ToolCallItem) ---
{
  "arguments": "{\"query\":\"coffee shop\",\"city\":\"Hamra, Beirut\",\"limit\":4}",
  "call_id": "call_KXLynO7COx3wZOKtXTbwtkFi",
  "name": "search_poi",
  "type": "function_call",
  "id": "fc_0fae348b3094b7db00691901cc468c819cad79173652d0b3cb",
  "status": "complete